In [1]:
%load_ext autoreload
%autoreload 2
from keras.applications.vgg16 import VGG16

import numpy as np
import os
import pandas as pd
from sklearn.metrics import accuracy_score

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
from keras.preprocessing import image
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

import models.simpleConv as simpleConv
# import models.betterConv as betterConv
from imgaug import augmenters as iaa

import h5py
from imgaug import augmenters as iaa
from keras.utils import np_utils

import utils
import preprocessing as pp
pp.init()

allTrainingFolders = [x for x in os.listdir(pp.trainPath) if x.startswith('n')]
trainingFolders = allTrainingFolders[0:5]

Using TensorFlow backend.


Checking to make sure output directories are created..
..done


In [ ]:
encoder = LabelBinarizer()
##MAKE SURE classLabels is set to ALL the folders you will train on, even if doing in batches
encoder = encoder.fit(trainingFolders) 

parse = False
train = True
saveModel = False
includeAugmented = False
onlyAugmented = False

augments = [iaa.GaussianBlur(3.0)]
h5filename = 'all.h5'

if parse:
    if os.path.isfile(os.path.join(pp.h5Path, h5filename)):
        os.remove(os.path.join(pp.h5Path, h5filename))
    pp.parseImages(trainingFolders, h5filename)
    
#NOTE: KEEP BATCH SIZE = # of all folders for now because we need to shuffle H5. 


callbacks = []
epochs = 5
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model = simpleConv.build_model(len(encoder.classes_))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


h5file = os.path.join(pp.h5Path, h5filename)


batch_size = 64

##FIT MODEL
if train:
    train_generator = pp.DataGenerator(h5file, trainingFolders, batch_size=64)
    print(len(train_generator))
    results = model.fit_generator(train_generator, epochs=epochs, callbacks = callbacks, verbose=1,
                              use_multiprocessing=False, workers=2)

    if saveModel:
        count = 1
        outputName = ""
        while True:
            k = ''
            if includeAugmented:
                k = 'with-aug'
            if onlyAugmented:
                k = 'aug-only'
            s = pp.baseModelName.format(epoch, count, k) + ".h5" 
            if not s in os.listdir(pp.outputModelPath):         
                outputName = s         
                break     
            else:         
                count+=1 
        print(outputName)
        model.save(os.path.join(pp.outputModelPath, outputName))
